# Evaluate the existing topics
Review misspelled words
Calculate Coherance Score

In [217]:
import sys
import os

module_path = os.path.abspath(os.path.join(".."))
if module_path not in sys.path:
    sys.path.append(module_path+"/scripts")
    sys.path.append(module_path+"/classes")

review_corpus_path = "../data/tech_review_sent_corpus.pkl"
vocab_path = "../data/glove-tech-revew-vocab.txt"
emb_filename = '../models/w2v_embedding'
aspect_file_path = "../results/aspect.json"
aspect_model_path = "../results/model_param"
vocab_path = "../data/vocab-text-review.txt"



In [366]:
import json

with open("../results/ABAE/test-abae-k-15-orth-0.8.json") as f:
    aspects = json.load(f)
    
print(aspects.keys())

dict_keys(['Aspect 0', 'Aspect 1', 'Aspect 2', 'Aspect 3', 'Aspect 4', 'Aspect 5', 'Aspect 6', 'Aspect 7', 'Aspect 8', 'Aspect 9', 'Aspect 10', 'Aspect 11', 'Aspect 12', 'Aspect 13', 'Aspect 14'])


Get topics words

In [367]:
aspect_jara = []
for aspect, words in aspects.items():   
    data = {}
    print(aspect)
    for word, score in words.items():
        for alt_word, alt_score in words.items():
            score = jaro_winkler_similarity(word, alt_word)
            if (score >= .9) & (score < 1):
                try:
                    data[word]["alt_words"][alt_word] = score
                except:
                    data[word] = {
                        "aspect":aspect,
                        "alt_words": {}
                    }
                data[word]["alt_words"][alt_word] = score
    aspect_jara.append(data)

Aspect 0
Aspect 1
Aspect 2
Aspect 3
Aspect 4
Aspect 5
Aspect 6
Aspect 7
Aspect 8
Aspect 9
Aspect 10
Aspect 11
Aspect 12
Aspect 13
Aspect 14


In [414]:
col_1 = []
col_2 = []
col_3 = []
for i, data in enumerate(aspect_jara):
    for key, alt in data.items():
        for word in alt["alt_words"].items():
            col_1.append(key)
            col_2.append(word[0])
            col_3.append(alt["aspect"])
            

df = pd.DataFrame([col_1, col_2, col_3]).T.rename(columns={0:"col_1",1:"col_2", 2:"col_3"})
df = df.sort_values(by=["col_3","col_1"])
print(df.shape)

# for i, row in df.iterrows():
#     print( '"' + str(row.col_1) +'": "' +str(row.col_2) +'",')

(204, 3)


In [296]:
# vocab = []
# for aspect, words in aspects.items():
#     for word, score in words.items():
#         vocab.append(word)

# vocab = list(set(vocab))
# vocab.sort()

# from nltk.metrics.distance import edit_distance, jaro_winkler_similarity

# data = {}
# for i, word in enumerate(vocab):
#     for j, alt_word in enumerate(vocab):
#         score = jaro_winkler_similarity(word, alt_word)
#         if (score >= .9) & (score < 1):
#             try:
#                 data[word][alt_word] = score
#             except:
#                 data[word] = {}
#                 data[word][alt_word] = score

# Create coherance score

In [444]:
import pandas as pd
import pickle

review_corpus_path = "../data/tech_review_sent_corpus.pkl"

with open(review_corpus_path,"rb") as f:
    tech_review_corpus = pickle.load(f)

print(len(tech_review_corpus))

148446


In [396]:
import importlib
importlib.reload(ABAE.reader)

<module 'ABAE.reader' from '/Users/christopherballenger/Code/smu-ds-opinion-mining-co-reviews/classes/ABAE/reader.py'>

In [445]:
import ABAE.reader as dataset

import json
from tqdm import tqdm
from multiprocessing import Pool

with open("../data/stop_words.json", "r") as f:
  stop_words = json.load(f)

def removeStopWords(review):
  tokens = review.split(" ")
  return " ".join([ word for word in tokens if word not in stop_words ])

reviews = pd.DataFrame(tech_review_corpus).review.tolist()

with Pool() as p:
  reviews = list(tqdm(p.imap(removeStopWords, reviews), total=len(reviews)))

vocab, train_x, overall_maxlen = dataset.get_data(reviews, vocab_path, vocab_size=0, maxlen=115)

vocab_inv = {}
for w, ind in vocab.items():
    vocab_inv[ind] = w
    
processed_docs = []
for review in train_x:
    processed_docs.append([vocab_inv[i] for i in review ])
    
print(len(train_x))

100%|██████████| 148446/148446 [00:06<00:00, 22169.72it/s]


 Creating vocab ...
   963539 total words, 22093 unique words
 Reading dataset ...
  train set
Corpus Size 148446
Total Document Analyzed 148446
<num> hit rate: 1.22%, <unk> hit rate: 0.00%
148446


In [400]:
from datetime import datetime
from gensim.corpora.dictionary import Dictionary

dictionary = Dictionary(processed_docs)
dictionary.add_documents([["<num>","<unk>"]])

corpus = [dictionary.doc2bow(text) for text in processed_docs]

In [408]:
[ review for review in processed_docs if "therein" in review ]




[['management',
  'management',
  'wa',
  'fair',
  'impartial',
  'regard',
  'request',
  'assistance',
  'emergency',
  'time',
  'personally',
  'learn',
  'latter',
  'true',
  'co',
  'work',
  'diverse',
  'environment',
  'everyone',
  'brought',
  'unique',
  'trait',
  'table',
  'knowledge',
  'wa',
  'easilly',
  'shared',
  'one',
  'technician',
  'another',
  'hard',
  'part',
  'job',
  'trying',
  'keep',
  'flow',
  'moving',
  'resolve',
  'issue',
  'forward',
  'support',
  'without',
  'going',
  'past',
  'estimated',
  'time',
  'limit',
  '<num>',
  'minute',
  'enjoy',
  'part',
  'love',
  'could',
  'fix',
  'issue',
  'customer',
  'therein',
  'true',
  'satisfactory',
  'technician',
  'know',
  'wa',
  'issue',
  'part',
  'make',
  'customer',
  'easier',
  'get',
  'duty',
  'completed']]

In [409]:
from datetime import datetime
from gensim.models import CoherenceModel
import numpy as np

def calc_coherance(topics):
    u_mass = []
    flags = []
    for n, topic in enumerate(topics):
        cm = CoherenceModel(topics=[topic], corpus=corpus, dictionary=dictionary, coherence='u_mass')
        u_mass.append(cm.get_coherence())
        return np.mean(u_mass)

In [410]:
import json
import os

files = [x for x in os.walk("../results/ABAE")]

scores = []
for t in files[0][2]:
    if (t[:6] == "abae-k") & (t[-4:] == 'json') :
        with open(os.path.join("../results/ABAE",t)) as f:
            aspects = json.load(f)

        topics = [[ word for word, score in words.items() ] for aspect, words in aspects.items() ]

        s = calc_coherance(topics)
        scores.append((t, s))

In [411]:
result = []
for score in scores:
    x = score[0][:-5].split("-")
    result.append({
        "k":x[2],
        "ortho":x[4],
        "score":score[1]
    })

In [412]:
import pandas as pd

pd.DataFrame(result).to_csv("../data/abae_umass.csv")

In [413]:
pd.DataFrame(result)

,k,ortho,score
0,9,1,-15.745435
1,15,0.8,-15.876360
2,9,0.5,-16.937292
3,11,0.8,-16.039723
4,12,0.5,-17.109498
5,11,0.5,-12.992026
6,5,1,-8.335060
7,9,0.8,-16.989177
8,15,0.5,-16.754466
9,7,0.3,-13.089316


LDA Visualize

In [213]:
import json

k = 5
ortho = 0.8
with open("../results/ABAE/abae-k-{0}-orth-{1}.json".format(str(k), str(ortho)), "r") as f:
    apsects = json.load(f)
    
topics = [[ word for word, score in words.items() ] for aspect, words in apsects.items() ]

topics_idx = [[ vocab[word] for word, score in words.items() ] for aspect, words in apsects.items() ] 

len(topics_idx)

5

In [214]:
from collections import Counter

vocab = Counter()

for topic in topics:
    for word in topic:
        vocab[word] += 1

In [216]:
vocab.most_common(10)

[('awesome', 2),
 ('great', 2),
 ('wonderful', 2),
 ('good', 2),
 ('management', 2),
 ('funny', 1),
 ('friendly', 1),
 ('really', 1),
 ('kind', 1),
 ('coworker', 1)]

In [74]:
from collections import Counter

def termFreq(doc):
    vocab = Counter()
    for word in doc:
        vocab[word] += 1
        
    return dict(vocab)

reviews_tf = list(map(termFreq, train_x))

In [195]:
import numpy as np
from tqdm import tqdm

document_sums = np.zeros((len(topics_idx),k))

def countWords(topic, doc):
    x = 0
    for w in topic:
        try:
            x += doc[w]
        except:
            pass
    return x

for row, topic in tqdm(enumerate(topics_idx), total=len(topics_idx)):
    for col, doc in enumerate(reviews_tf):
        ct = countWords(topic, doc)
        document_sums[row,col] = ct

alpha = 1/k
theta = []
for i in range(k):
    if document_sums[i,:].sum() == 0: break
    theta.append(( document_sums[i,:] + alpha ) / document_sums[i,:].sum())
    
theta = np.array(theta).T

print("Theta Shape", theta.shape)

100%|██████████| 5/5 [00:11<00:00,  2.32s/it]

Theta Shape (148425, 5)


In [204]:
topic_sums = np.zeros((len(vocab), k))

def countWords(topic, doc):
    x = 0
    for w in topic:
        try:
            x += doc[w]
        except:
            pass
    return x

for row, topic in tqdm(enumerate(topics_idx), total=len(topics_idx)):
    for col, doc in enumerate(reviews_tf):
        ct = countWords(topic, doc)
        document_sums[row,col] = ct

(5, 22791)

In [ ]:
# https://github.com/derekgreene/topic-model-tutorial/blob/master/3%20-%20Parameter%20Selection%20for%20NMF.ipynb
import gensim
from itertools import combinations
from scipy.spatial.distance import cosine

w2v_model = gensim.models.Word2Vec.load(emb_filename)

def getVector(w2v_model, word):
    try:
        v = w2v_model.wv[word]
    except:
        v = np.repeat(.00000001, w2v_model.vector_size)
    return v

def calculate_coherence( w2v_model, term_rankings ):
    overall_coherence = 0.0
    for topic_index in range(len(term_rankings)):
        # check each pair of terms
        pair_scores = []
        for pair in combinations( term_rankings[topic_index], 2 ):
            pair_scores.append( 1- cosine(getVector(w2v_model, pair[0]), getVector(w2v_model,pair[1])) )
        # get the mean for all pairs in this topic
        topic_score = sum(pair_scores) / len(pair_scores)
        overall_coherence += topic_score
    # get the mean score across all topics
    return overall_coherence / len(term_rankings)

calculate_coherence(w2v_model, topics)

In [300]:
from nltk import WordNetLemmatizer

lmtzr = WordNetLemmatizer()

In [308]:
lmtzr.lemmatize("nights")

'night'